# Criação do modelo de classificação para o chatbot

## Importando os módulos necessários:

In [1]:
!pip install tensorflow
!pip install --upgrade tensorflow_text
!pip install --upgrade tensorflow_hub
!pip install pandas
!pip install tflearn
!pip install sklearn
!pip install numpy

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from tensorflow import keras

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 11.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.7 MB/s eta 0:00:0000:01
  Using cached protobuf-4.22.3-cp37-abi3-manylinux2014_x86_64.whl (302 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 14.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 2.7 MB/s eta 0:00:00:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 KB 6.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 25.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━

2023-05-02 16:01:59.768501: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-02 16:01:59.799206: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-02 16:01:59.800019: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 16:02:02.065730: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Usando uma build pré-treinada e iniciando modelo

+ Usamos um modelo pré-treinado do Tensorflow Hub
+ Usamos o pré-processador equivalente ao modelo escolhido
+ Durante a execução, alguns warnings são exibidos. Isto se dá pela falta de uma GPU na máquina, então o aviso pode ser ignorado.

In [2]:
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)


2023-05-02 16:02:05.387302: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-02 16:02:05.387707: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-02 16:02:05.387828: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-02 16:02:05.388183: I tensorflow/core/

## Importando os dados para treinamento

+ Os dados devem estar armazenados no mesmo diretório que o código
+ Serão armazenados na forma de Dataframe, usando Pandas
+ Após importação, é feita a transformação da variável categórica "Tag"

In [3]:
df = pd.read_csv("data.csv")

df['funny'] = df['Tag'].apply(lambda x: 1 if x=='funny' else 0)
df['goodbye'] = df['Tag'].apply(lambda x: 1 if x=='goodbye' else 0)
df['greeting'] = df['Tag'].apply(lambda x: 1 if x=='greeting' else 0)
df['me'] = df['Tag'].apply(lambda x: 1 if x=='me' else 0)
df['query'] = df['Tag'].apply(lambda x: 1 if x=='query' else 0)
df['thanks'] = df['Tag'].apply(lambda x: 1 if x=='thanks' else 0)
df['use'] = df['Tag'].apply(lambda x: 1 if x=='use' else 0)

df.head(1)

,Qual,Tag,funny,goodbye,greeting,me,query,thanks,use
0,Você pode me contar uma piada?,funny,1,0,0,0,0,0,0


## Criação de dados de treinamento

Nesta etapa, os dados salvos no dataframe são separados em duas partes:

+ Uma lista de frases, seriam os dados de entrada
+ Uma lista contendo a categoria (transformada em uma lista) equivalente a cada frase.

Cada categoria é representada por uma lista de zeros contendo apenas um número um.

In [4]:
list = df.values.tolist()
intents = []
sents = []
for i in range(len(list)):
    line = list[i][2:]
    intents.append(line)
    sents.append(list[i][0])

## Criando o modelo

+ A camada de input receberá um texto e passará adiante este texto pré-processado pelo modelo retirado do Tensorflow Hub
+ Este texto pré-processado é processado pelas camadas do bert, modelo extraído do Tensorflow Hub
+ Para evitar overfitting, foi adicionada uma camada de dropout
+ Após o dropout, foi feita uma camada de classificação. Ela possui 7 neurônios, pois o sistema possui 7 classificações distintas. A função de ativação escolhida foi a softmax pois esta se sai melhor para tarefas de classificação com múltiplas categorias.
+ O modelo é então criado com a arquitetura definida.

In [5]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess_model(text_input)
outputs = bert_model(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(7, activation='softmax', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])


2023-05-02 16:02:11.793406: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string and shape [?]
	 [[{{node inputs}}]]
2023-05-02 16:02:11.809963: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype string and shape [?]
	 [[{{node Placeholder}}]]
2023-05-02 16:02:11.913477: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype int32 and shape [?,128]
	 [[{{node inputs}}]]
2023-05-02 1

## Definidas as métricas do modelo e feita a sua compilação

In [6]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

## O modelo é então treinado

+ Usa-se como conjunto de treinamento as frases retiradas do dataset
+ Como trata-se de uma operação de aprendizagem supervisionada, as etiquetas para cada entrada são dadas pela categoria transformada anteriormente.
+ usamos 1000 épocas e um tamanho de batch de 10.

In [7]:
hist = model.fit(sents,intents,epochs=1000, batch_size=10)


Epoch 1/1000


2023-05-02 16:02:14.853055: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/StatefulPartitionedCall_3' with dtype float and shape [?,?,768]
	 [[{{node gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/StatefulPartitionedCall_3}}]]
2023-05-02 16:02:14.853128: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/StatefulPartitionedCall_4' with dtype float and shape [?,?,768]
	 [[{{node gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/StatefulPartitionedCall_4}}]]
2023

12/12 [==============================] - 15s 806ms/step - loss: 0.5231 - accuracy: 0.8522 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 2/1000
12/12 [==============================] - 9s 753ms/step - loss: 0.4269 - accuracy: 0.8571 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 3/1000
12/12 [==============================] - 9s 749ms/step - loss: 0.4075 - accuracy: 0.8534 - precision: 0.3333 - recall: 0.0261
Epoch 4/1000
12/12 [==============================] - 9s 759ms/step - loss: 0.3999 - accuracy: 0.8596 - precision: 0.7500 - recall: 0.0261
Epoch 5/1000
12/12 [==============================] - 9s 759ms/step - loss: 0.3853 - accuracy: 0.8584 - precision: 0.6000 - recall: 0.0261
Epoch 6/1000
12/12 [==============================] - 9s 763ms/step - loss: 0.3887 - accuracy: 0.8584 - precision: 0.6667 - recall: 0.0174
Epoch 7/1000
12/12 [==============================] - 9s 751ms/step - loss: 0.3802 - accuracy: 0.8584 - precision: 0.6667 - recall: 0.0174
Epoch 8/1000
12/12 [===

## O modelo é então salvo

+ Foi utilizado o modelo no formato hdf5

In [8]:
model.save('chatModel.h5')

2023-05-02 18:38:41.934668: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
